## Final Machine Learning Pipeline
The pipeline features: 

* open source classes
* in-house package classes
* only uses the selected features

### Rossman Store Sales Prediction
The aim of the project is to build an end-to-end machine learning model to predict the sales of a given store and a set of inputs, including the promotions, competition, school and state holidays, seasonality, and locality.